In [1]:
using Pkg
Pkg.activate("."); Pkg.instantiate()
Pkg.precompile()

using CSV
using DataFrames
using Plots
using Glob
using Dates

  Activating project at `~/proyectos/TFG-ProduccionElectrica/codigo/Julia`


In [2]:
filepath = "../data/predict-energy-behavior-of-prosumers"
files = glob("*.csv", filepath)

8-element Vector{String}:
 "../data/predict-energy-behavior-of-prosumers/client.csv"
 "../data/predict-energy-behavior-of-prosumers/county_lon_lats.csv"
 "../data/predict-energy-behavior-of-prosumers/electricity_prices.csv"
 "../data/predict-energy-behavior-of-prosumers/forecast_weather.csv"
 "../data/predict-energy-behavior-of-prosumers/gas_prices.csv"
 "../data/predict-energy-behavior-of-prosumers/historical_weather.csv"
 "../data/predict-energy-behavior-of-prosumers/train.csv"
 "../data/predict-energy-behavior" ⋯ 20 bytes ⋯ "r_station_to_county_mapping.csv"

In [3]:
dfs = Dict(
    splitext(basename(archivo))[1] => CSV.read(archivo, DataFrame)
    for archivo in files
)
dfs["weather_station"] = dfs["weather_station_to_county_mapping"]
delete!(dfs, "weather_station_to_county_mapping")

Dict{String, DataFrame} with 8 entries:
  "historical_weather" => 1710802×18 DataFrame…
  "electricity_prices" => 15286×4 DataFrame…
  "weather_station"    => 112×4 DataFrame…
  "client"             => 41919×7 DataFrame…
  "train"              => 2018352×9 DataFrame…
  "forecast_weather"   => 3424512×18 DataFrame…
  "county_lon_lats"    => 75×4 DataFrame…
  "gas_prices"         => 637×5 DataFrame…

In [51]:
as = rename(
    dfs["electricity_prices"],
    :forecast_date => :datetime
);

formato = DateFormat("yyyy-mm-dd HH:MM:SS");

as.datetime = DateTime.(as.datetime, formato);
as.date = Date.(as.datetime)
as.year = Dates.year.(as.datetime)
as.quarter = Dates.quarter.(as.datetime)
as.month = Dates.month.(as.datetime)
as.day_of_year = Dates.dayofyear.(as.datetime)
as.day_of_month = Dates.dayofmonth.(as.datetime)
as.day_of_week = Dates.dayofweek.(as.datetime)

as

Row,datetime,euros_per_mwh,origin_date,data_block_id,date,year,quarter,month,day_of_year,day_of_month,day_of_week
,DateTime,Float64,String31,Int64,Date,Int64,Int64,Int64,Int64,Int64,Int64
1,2021-09-01T00:00:00,92.51,2021-08-31 00:00:00,1,2021-09-01,2021,3,9,244,1,3
2,2021-09-01T01:00:00,88.9,2021-08-31 01:00:00,1,2021-09-01,2021,3,9,244,1,3
3,2021-09-01T02:00:00,87.35,2021-08-31 02:00:00,1,2021-09-01,2021,3,9,244,1,3
4,2021-09-01T03:00:00,86.88,2021-08-31 03:00:00,1,2021-09-01,2021,3,9,244,1,3
5,2021-09-01T04:00:00,88.43,2021-08-31 04:00:00,1,2021-09-01,2021,3,9,244,1,3
6,2021-09-01T05:00:00,93.58,2021-08-31 05:00:00,1,2021-09-01,2021,3,9,244,1,3
7,2021-09-01T06:00:00,118.7,2021-08-31 06:00:00,1,2021-09-01,2021,3,9,244,1,3
8,2021-09-01T07:00:00,135.44,2021-08-31 07:00:00,1,2021-09-01,2021,3,9,244,1,3
9,2021-09-01T08:00:00,140.72,2021-08-31 08:00:00,1,2021-09-01,2021,3,9,244,1,3


In [17]:
DateTime.(dfs["gas_prices"].forecast_date)

637-element Vector{DateTime}:
 2021-09-01T00:00:00
 2021-09-02T00:00:00
 2021-09-03T00:00:00
 2021-09-04T00:00:00
 2021-09-05T00:00:00
 2021-09-06T00:00:00
 2021-09-07T00:00:00
 2021-09-08T00:00:00
 2021-09-09T00:00:00
 2021-09-10T00:00:00
 ⋮
 2023-05-22T00:00:00
 2023-05-23T00:00:00
 2023-05-24T00:00:00
 2023-05-25T00:00:00
 2023-05-26T00:00:00
 2023-05-27T00:00:00
 2023-05-28T00:00:00
 2023-05-29T00:00:00
 2023-05-30T00:00:00

In [6]:
dfs["gas_prices"]["forecast_date"]

ArgumentError: ArgumentError: syntax df[column] is not supported use df[!, column] instead